In [42]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

In [2]:
baseUrl = 'https://api.polygon.io/v2'
apiKey = 'LQeVT4355IbDNjGP2HKvAFlagtPfJPL4'

In [3]:
def request_stock(ticker, start, end):
    url = f'{baseUrl}/aggs/ticker/{ticker}/range/1/day/{start}/{end}?apiKey={apiKey}'
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Request Stock Error {resp.status_code}")
        return None
    return resp.json()

In [4]:
def get_csv(tickers):
    rsl = {}
    for i in range(len(tickers)):
        if i > 0 and i % 5 == 0:
            print("wait 60 secs...")
            time.sleep(60)
        resp = request_stock(tickers[i], '2022-01-01', '2024-12-31')
        df = pd.DataFrame(resp['results'])
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        df.columns = ['volumn', 'vwap', 'open', 'close', 'high', 'low', 'time', 'num_trades']
        rsl[tickers[i]] = df
    return rsl

In [5]:
tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'MSFT', 'IBM', 'ORCL', 'NVDA', 'INTC']

In [6]:
stock_dfs = get_csv(tickers)

wait 60 secs...


In [8]:
for key, df in stock_dfs.items():
    df.to_csv(f'dataset/stocks/{key}.csv', index=False)

In [39]:
def request_news(ticker, end='2024-12-31'):
    url = f'{baseUrl}/reference/news?ticker={ticker}&published_utc.lte={end}&limit=1000&apiKey={apiKey}'
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Request News Error {resp.status_code}")
        return None
    return resp.json()

In [53]:
def get_news(ticker):
    main_list = []
    relation_list = []
    last_date = '2024-12-31'
    for i in range(100):
        if i > 0 and i % 5 == 0:
            print("wait 60 secs...")
            time.sleep(60)
        raw_data = request_news(ticker, last_date)['results']
        last_date = raw_data[-1]['published_utc'][:10]
        
        for item in raw_data:
            if item['published_utc'].startswith(last_date):
                break
            item['ticker'] = ticker
            item['publish_time'] = datetime.fromisoformat(item['published_utc'][:-1])
            item['publisher'] = item['publisher']['name'] if 'publisher' in item else ""
            item['keywords'] = ",".join(item['keywords']) if 'keywords' in item else ""
            
            main_cols = ['id', 'publish_time', 'title', 'article_url', 
                         'ticker', 'publisher', 'description', 'keywords']
            main_list.append({k: item[k] if k in item else "" for k in main_cols})
            
            if 'insights' in item:
                for relation in item['insights']:
                    relation['news_id'] = item['id']
                    relation['source_ticker'] = item['ticker']
                    relation['time'] = item['publish_time']
                    relation_list.append(relation)
        if len(raw_data) < 1000 or last_date < '2022-11-01':
            break
    main_df = pd.DataFrame(main_list)
    relation_df = pd.DataFrame(relation_list)
    return main_df, relation_df

In [63]:
news_main_dfs = {}
news_relation_dfs = {}
for i in range(len(tickers)):
    main_df, relation_df = get_news(tickers[i])
    news_main_dfs[tickers[i]] = main_df
    news_relation_dfs[tickers[i]] = relation_df
    if i < len(tickers) - 1:
        print("wait 60 secs...")
        time.sleep(60)

wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...
wait 60 secs...


In [64]:
for key, df in news_main_dfs.items():
    df.to_csv(f'dataset/news/{key}_main.csv', index=False)

In [65]:
for key, df in news_relation_dfs.items():
    df.to_csv(f'dataset/news/{key}_relation.csv', index=False)